<a href="https://colab.research.google.com/github/liminovna/HSE_NLP_ASSIGNMENTS/blob/main/module2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [ ]:
! wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/wiki_data.txt.zip
!unzip -o wiki_data.txt.zip

--2024-11-24 18:23:11--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/wiki_data.txt.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/wiki_data.txt.zip [following]
--2024-11-24 18:23:11--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/wiki_data.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18709143 (18M) [application/zip]
Saving to: ‘wiki_data.txt.zip’

wiki_data.txt.zip   100%[===================>]  17.84M  --.-KB/s    in 0.1s    

2024-11-24 18:23:11 (148 MB/s) - ‘wiki_data.tx

In [ ]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [ ]:
!pip install textdistance

In [ ]:
from collections import Counter
import re
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [ ]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [ ]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    # вся эффективноть берется из того, что мы сразу считаем близость
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    # <добавленный код>
    best = []

    max_metric = closest[0][1]

    for pair in closest:
      if pair[1] == max_metric:
        best.append(pair)
      else:
        break

    res = []
    max_p = 0

    for pair in best:
      prob = P(pair[0])
      if prob > max_p:
        res=((pair + (prob,)))
        max_p = prob

    # возвращаем только 1 вариант с наибольшей вероятностью в нашем корпусе.
    # Помимо самого кандидата и similarity также возвращается вероятность
    return res
    # </добавленный код>

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!)

3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [1]:
! wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/wiki_data.txt.zip
!unzip -o wiki_data.txt.zip

--2024-11-29 17:01:16--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/wiki_data.txt.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/wiki_data.txt.zip [following]
--2024-11-29 17:01:16--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/wiki_data.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18709143 (18M) [application/zip]
Saving to: ‘wiki_data.txt.zip’

wiki_data.txt.zip   100%[===================>]  17.84M  80.8MB/s    in 0.2s    

2024-11-29 17:01:17 (80.8 MB/s) - ‘wiki_data.t

In [2]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [3]:
!pip install textdistance

In [4]:
from collections import Counter
import re
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [5]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [6]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [7]:
def deletions1(word):
    assert len(word) > 1
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

In [92]:
correct_dict = dict()

for word, freq in vocab.most_common():
  # print(word, freq)
  if len(word) > 1:
    delit = deletions1(word)
    # print(delit)
    for s in delit:
      curr = correct_dict.get(s)
      # print(s, curr)
      if not curr:
        # записываем с словарь кандидатов варианты только для того, слова, которое встретилось чаще
        correct_dict.update({s: (word, freq)})
        # print('added', {s: (word, freq)})
  # else:
  #   correct_dict[word] = (word, freq)

In [91]:
for word, freq in vocab.most_common():
  if 'з' in word and len(word)<4:
    print(word, freq)

из 29150
за 17971
раз 3388
без 1992
зал 211
з 175
газ 105
зао 81
фз 77
миз 75
зон 53
оз 53
баз 51
зс 51
каз 43
зла 42
вуз 41
виз 40
паз 38
зис 38
изд 37
зоя 35
роз 35
зуб 33
зои 32
зд 29
зло 26
зуд 26
коз 26
зак 25
маз 25
фаз 24
сзм 24
зо 21
муз 21
зов 20
зав 20
цзе 20
изо 20
зпу 19
воз 19
ваз 17
зам 17
цзы 17
озу 17
аз 17
низ 16
зил 16
зко 16
цзи 15
ямз 15
азс 14
зе 14
экз 13
зее 13
сза 12
фзу 12
киз 12
доз 11
лиз 11
зиф 11
зол 9
уз 9
аза 9
зря 9
дзе 9
зи 9
ссз 9
зум 8
зад 8
зрк 8
исз 8
звп 8
дмз 8
пиз 8
зна 7
зм 7
кшз 7
тюз 7
зою 7
яз 7
язы 7
кэз 7
рза 6
узи 6
вяз 6
язв 6
злу 6
зу 6
зан 6
дез 6
овз 6
кмз 6
знб 6
цзо 6
кыз 6
кз 5
мтз 5
зим 5
юз 5
вёз 5
язь 5
нпз 5
риз 5
хкз 5
срз 5
таз 5
изб 5
тэз 5
зпо 5
мзв 5
дзё 4
мз 4
зиу 4
заз 4
зап 4
пзу 4
змс 4
азо 4
дзи 4
цвз 4
хоз 4
зия 4
зпт 4
зии 3
хтз 3
пкз 3
цзю 3
зя 3
уаз 3
луз 3
зы 3
изз 3
уза 3
фзо 3
дэз 3
двз 3
туз 3
азы 3
лез 3
цзу 3
нмз 3
зец 3
зур 3
шпз 3
дуз 3
цзя 3
рхз 3
зос 2
гмз 2
нез 2
тоз 2
гиз 2
изъ 2
заб 2
зья 2
зем 2
руз 2

In [96]:
def choose_best(misspelled_word):
  assert len(misspelled_word) > 1, 'Слишком короткое слово'
  options = deletions1(misspelled_word)
  # print(options)
  max_likelihood = 0
  likely_word = misspelled_word
  for o in options:
    # print(o)
    curr_pair = correct_dict.get(o)
    print(curr_pair)
    if correct_dict.get(o):
      if curr_pair[1] > max_likelihood:
        max_likelihood = curr_pair[1]
        likely_word = curr_pair[0]
  return likely_word

In [69]:
# Если слово только из одной буквы, выдается ошибка
choose_best('а')

AssertionError: Слишком короткое слово

In [97]:
choose_best('тз')

('от', 18527)
('из', 29150)


'из'

In [101]:
choose_best('гдн')

('дня', 736)
('ген', 28)
('где', 5760)


'где'